# Clean data

In [1]:
!pip install nltk
!pip install sklearn

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()
print(spark)

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-df392e77-c241-40b0-98e4-0c0a5ce29133;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 146ms :: artifacts dl 8ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   4   |   0   |   0   |   0   ||   4   |   0   |
	---------------------------------------------------------------------
:: retrieving ::

In [7]:
crypto_df = spark.read.format("mongo").option("uri","mongodb://mongo/seng550.crypto").load()
crypto_df.show()
headers_df = spark.read.format("mongo").option("uri","mongodb://mongo/seng550.headers").load()
headers_df.show()

+--------------------+------------------+----------+------------------+
|                 _id|             close|      date|              open|
+--------------------+------------------+----------+------------------+
|{619f20a05fec62ac...|        61238.6222|2021-10-23| 60690.28731420007|
|{619f20a15fec62ac...| 60694.00000650834|2021-10-22| 62196.96438572152|
|{619f20a15fec62ac...|62298.093007718286|2021-10-21| 65961.04976056097|
|{619f20a15fec62ac...| 65986.30293718613|2021-10-20|        64252.7263|
|{619f20a15fec62ac...|        64217.0839|2021-10-19|        62020.0126|
|{619f20a15fec62ac...|        61953.2331|2021-10-18| 61543.26469764322|
|{619f20a15fec62ac...| 61582.25155249131|2021-10-17| 60867.89483798279|
|{619f20a15fec62ac...| 60860.89183706596|2021-10-16| 61698.03021926293|
|{619f20a15fec62ac...| 61526.33413298988|2021-10-15|57312.383791730215|
|{619f20a15fec62ac...|  57351.7384160736|2021-10-14|        57385.8884|
|{619f20a15fec62ac...|        57357.6067|2021-10-13| 56030.57101

### Clean the crypto dataframe - JD

In [4]:
# https://stackoverflow.com/questions/58823628/call-a-function-for-each-row-of-a-dataframe-in-pysparknon-pandas
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
def get_change(close, open_):
    return  close - open_
spark.udf.register("change_udf", get_change)
# crypto_df.show()
crypto_df = crypto_df.withColumn('change', get_change(col('close'), col('open')))
crypto_df.show()

+--------------------+------------------+----------+------------------+-------------------+
|                 _id|             close|      date|              open|             change|
+--------------------+------------------+----------+------------------+-------------------+
|{619f20a05fec62ac...|        61238.6222|2021-10-23| 60690.28731420007|  548.3348857999299|
|{619f20a15fec62ac...| 60694.00000650834|2021-10-22| 62196.96438572152|-1502.9643792131828|
|{619f20a15fec62ac...|62298.093007718286|2021-10-21| 65961.04976056097| -3662.956752842685|
|{619f20a15fec62ac...| 65986.30293718613|2021-10-20|        64252.7263| 1733.5766371861318|
|{619f20a15fec62ac...|        64217.0839|2021-10-19|        62020.0126|  2197.071299999996|
|{619f20a15fec62ac...|        61953.2331|2021-10-18| 61543.26469764322|  409.9684023567752|
|{619f20a15fec62ac...| 61582.25155249131|2021-10-17| 60867.89483798279|   714.356714508518|
|{619f20a15fec62ac...| 60860.89183706596|2021-10-16| 61698.03021926293| -837.138

### Clean the headers dataframe - JD

In [8]:

import string
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def clean_header(header):
    porter = PorterStemmer()
    # headers are small, so local process is ok
    header = header.strip()
    if not header:
        return ''
    header = header.translate(str.maketrans('', '', string.punctuation))
    return ' '.join(porter.stem(word) for word in header.split() if word not in ENGLISH_STOP_WORDS)
udf_clean_header = udf(lambda s: clean_header(s), StringType())
headers_df = headers_df.withColumn('clean_headline', udf_clean_header(col('headline')))
headers_df.show()

+--------------------+----------+--------------------+--------------------+
|                 _id|      date|            headline|        clean_header|
+--------------------+----------+--------------------+--------------------+
|{619f1214b82509cf...|2021-08-01|Family speaks out...|famili speak raci...|
|{619f1215b82509cf...|2021-08-01|China invasion of...|china invas taiwa...|
|{619f1215b82509cf...|2021-08-01|US digital paymen...|us digit payment ...|
|{619f1215b82509cf...|2021-08-01|‘Allow Malaysians...|‘allow malaysian ...|
|{619f1215b82509cf...|2021-08-01|Australia's After...|australia afterpa...|
|{619f1215b82509cf...|2021-08-01|Americans blast A...|american blast au...|
|{619f1215b82509cf...|2021-08-01|Turkey fires ‘lik...|turkey fire ‘like...|
|{619f1215b82509cf...|2021-08-01|Horoscope today, ...|horoscop today mo...|
|{619f1215b82509cf...|2021-08-01|Horoscope Today: ...|horoscop today as...|
|{619f1215b82509cf...|2021-08-01|Horoscope Today: ...|horoscop today as...|
|{619f146e97

### Join - Owen

### Split data - JD

In [10]:
# method 1 https://stackoverflow.com/a/51773836, can only be done on 1 partition
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

# assign ranks
# TODO better method to avoid this problem - if we do random splits it might be better? 
headers_df = headers_df.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("date")))
# split
train_headlines_df = headers_df.where("rank <= .8").drop("rank")
test_headlines_df = headers_df.where("rank > .8").drop("rank")

train_headlines_df.show()
test_headlines_df.show()

# method 2 https://stackoverflow.com/a/53193549, needs the number of dates fed in to split
# from pyspark.ml.feature import StringIndexer

# stringIndexer = StringIndexer(inputCol="date", outputCol="index")
# model = stringIndexer.fit(headers_df)
# headers_df = model.transform(headers_df).withColumn("index", col("index").cast("int"))

# headers_df.show()

21/12/06 20:37:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/06 20:37:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/06 20:37:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/06 20:37:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/06 20:37:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+----------+--------------------+--------------------+
|                 _id|      date|            headline|        clean_header|
+--------------------+----------+--------------------+--------------------+
|{61a29a055c39c628...|2021-03-01|Black Amazon exec...|black amazon exec...|
|{61a29a065c39c628...|2021-03-01|Jeff Bezos versus...|jeff bezo versu m...|
|{61a29a065c39c628...|2021-03-01|Michael Gudinski ...|michael gudinski ...|
|{61a29a065c39c628...|2021-03-01|New, revolutionar...|new revolutionari...|
|{61a29a065c39c628...|2021-03-01|Lady Gaga's dog w...|ladi gaga dog wal...|
|{61a29a065c39c628...|2021-03-01|Study assesses ca...|studi assess caus...|
|{61a29a065c39c628...|2021-03-01|Seahawks QB Russe...|seahawk qb russel...|
|{61a29a065c39c628...|2021-03-01|Calls for permane...|call perman reope...|
|{61a29a065c39c628...|2021-03-01|An insanely blurr...|an insan blurri t...|
|{61a29a065c39c628...|2021-03-01|One third of Hin ...|one hin leong fou...|
|{61a29a065c

21/12/06 20:37:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+----------+--------------------+--------------------+
|                 _id|      date|            headline|        clean_header|
+--------------------+----------+--------------------+--------------------+
|{619f14897cd306c4...|2021-09-08|How to Support A ...|how support a fri...|
|{619f14897cd306c4...|2021-09-08|Tropical Storm Mi...|tropic storm mind...|
|{619f14897cd306c4...|2021-09-08|Pediatric COVID c...|pediatr covid cas...|
|{619f14897cd306c4...|2021-09-08|McConnell sends '...|mcconnel send lot...|
|{619f14897cd306c4...|2021-09-08|Taliban cabinet i...|taliban cabinet i...|
|{619f14897cd306c4...|2021-09-08|Karnal stir to mi...|karnal stir mirro...|
|{619f14897cd306c4...|2021-09-08|Air New Zealand a...|air new zealand a...|
|{619f14897cd306c4...|2021-09-08|Labor Department ...|labor depart tout...|
|{619f14897cd306c4...|2021-09-08|COVID-19 breakthr...|covid19 breakthro...|
|{619f14897cd306c4...|2021-09-08|Scott Morrison se...|scott morrison se...|
|{619f148a7c

21/12/06 20:37:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
21/12/06 20:37:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+----------+--------------------+--------------------+--------------------+-----+
|                 _id|      date|            headline|        clean_header|                rank|index|
+--------------------+----------+--------------------+--------------------+--------------------+-----+
|{61a29a055c39c628...|2021-03-01|Black Amazon exec...|black amazon exec...|                 0.0|    0|
|{61a29a065c39c628...|2021-03-01|Jeff Bezos versus...|jeff bezo versu m...|                 0.0|    0|
|{61a29a065c39c628...|2021-03-01|Michael Gudinski ...|michael gudinski ...|                 0.0|    0|
|{61a29a065c39c628...|2021-03-01|New, revolutionar...|new revolutionari...|                 0.0|    0|
|{61a29a065c39c628...|2021-03-01|Lady Gaga's dog w...|ladi gaga dog wal...|                 0.0|    0|
|{61a29a065c39c628...|2021-03-01|Study assesses ca...|studi assess caus...|                 0.0|    0|
|{61a29a065c39c628...|2021-03-01|Seahawks QB Russe...|seahawk qb russel..

21/12/06 20:37:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
